In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

Cloning into 'Recommender-Systems-Challenge-2022'...
remote: Enumerating objects: 596, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 596 (delta 99), reused 127 (delta 52), pack-reused 404
Receiving objects: 100% (596/596), 98.86 MiB | 13.23 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Updating files: 100% (248/248), done.


In [2]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1553 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 940 ( 2.3%) Users that have less than 1 test interactions


In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(topK=585, l1_ratio=0.009929794168572358, alpha=0.0018687691383859428)

In [4]:
result_df, _ = evaluator_test.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM ElasticNet - MAP: {}".format(result_df.loc[10]["MAP"]))

EvaluatorHoldout: Processed 40689 (100.0%) in 53.60 sec. Users per second: 759
SLIM ElasticNet - MAP: 0.0


In [3]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(topK=585, l1_ratio=0.009929794168572358, alpha=0.0018687691383859428)

SLIMElasticNetRecommender: Processed 2951 (12.0%) in 5.00 min. Items per second: 9.83
SLIMElasticNetRecommender: Processed 6205 (25.3%) in 10.00 min. Items per second: 10.34
SLIMElasticNetRecommender: Processed 9532 (38.9%) in 15.00 min. Items per second: 10.59
SLIMElasticNetRecommender: Processed 12787 (52.2%) in 20.00 min. Items per second: 10.65
SLIMElasticNetRecommender: Processed 15900 (64.9%) in 25.00 min. Items per second: 10.60
SLIMElasticNetRecommender: Processed 18926 (77.2%) in 30.00 min. Items per second: 10.51
SLIMElasticNetRecommender: Processed 21926 (89.5%) in 35.00 min. Items per second: 10.44
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 39.22 min. Items per second: 10.41


## Submissions

In [5]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [6]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender_SLIMElasticNet.recommend(user, cutoff=10, remove_seen_flag=True))

In [7]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions/Submission_SLIM_EN_new.csv', index=False)